In [23]:
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dash
from dash import dcc
from dash import html

import statsmodels.api as sm

data_folder = Path("data").resolve()

In [24]:
# Read in the data
df = pd.read_csv(data_folder / "marketing_data.csv", sep=";")

In [25]:
df.date = pd.to_datetime(df.date, format="%Y-%m-%d")
df['cohort'] = df['site'] + '_' + df['country']

df = df.groupby(['cohort', 'date']).sum().reset_index()

In [26]:

df['conversion_per_session'] = df['conversions'] / df['sessions']
df['price_per_impression'] = df['total_spend'] / df['impressions']

In [27]:
df = df.groupby(['cohort', pd.Grouper(key='date', freq='D')])['total_spend', 'impressions', 'sessions', 'conversions', 'price_per_impression', 'conversion_per_session'].sum()

/var/folders/qd/2qwtnqg57r915dp6p3dq762c0000gn/T/ipykernel_6095/37315400.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [28]:
# Define a dash app
app = dash.Dash(__name__)

# Define the layout

app.layout = html.Div(children=[
    dcc.Graph(id='graph'),
])


In [29]:
import plotly.graph_objs as go

@app.callback(
    dash.dependencies.Output('graph', 'figure'),
    [dash.dependencies.Input('cohort-dropdown', 'value'),
     dash.dependencies.Input('feature-dropdown', 'value')])
def update_graph(cohort, feature):
    df_cohort = df.loc[cohort]
    series = df_cohort[feature]
    result = sm.tsa.seasonal_decompose(series, model='additive', period=30)

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=result.observed.index,
        y=result.observed.values,
        mode='lines',
        name='Observed'
    ))

    fig.add_trace(go.Scatter(
        x=result.trend.index,
        y=result.trend.values,
        mode='lines',
        name='Trend'
    ))

    fig.add_trace(go.Scatter(
        x=result.seasonal.index,
        y=result.seasonal.values,
        mode='lines',
        name='Seasonal'
    ))

    fig.update_layout(
        title=f'{feature} {cohort} Seasonal Decomposition',
        xaxis_title='Date',
        yaxis_title='Value'
    )

    return fig


In [30]:


# Define the dropdown menus
cohort_dropdown = dcc.Dropdown(
    id='cohort-dropdown',
    options=[{'label': cohort, 'value': cohort} for cohort in df.index.levels[0]],
    value=df.index.levels[0][0]
)

feature_dropdown = dcc.Dropdown(
    id='feature-dropdown',
    options=[{'label': feature, 'value': feature} for feature in df.columns],
    value=df.columns[0]
)

# Define the layout of the app
app.layout = html.Div(children=[
    html.H1(children='Seasonal Decomposition Dashboard'),
    html.Div(children='Select a cohort and a feature to see the seasonal decomposition.'),
    html.Div(children=[
        html.Label('Cohort'),
        cohort_dropdown
    ]),
    html.Div(children=[
        html.Label('Feature'),
        feature_dropdown
    ]),
    dcc.Graph(id='graph')
])

# Run the app
if __name__ == '__main__':
    app.run_server(port=8051)


Dash is running on http://127.0.0.1:8051/

 * Serving Flask app '__main__'
 * Debug mode: off


In [15]:


from dash.dependencies import Input, Output
cohort_options = [{'label': c, 'value': c} for c in df.cohort.unique()]
feature_options = [{'label': f, 'value': f} for f in df.drop(columns=['cohort', 'date']).columns]


# define app layout
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1("Dashboard"),
    html.Div([
        html.Label('Select Cohort:'),
        dcc.Dropdown(
            id='cohort-dropdown',
            options=cohort_options,
            value=df.cohort.unique()[0]
        )
    ]),
    html.Div([
        html.Label('Select Feature:'),
        dcc.Dropdown(
            id='feature-dropdown',
            options=feature_options,
            value='total_spend'
        )
    ]),
    html.Div([
        dcc.Graph(id='graph1'),
        dcc.Graph(id='graph2'),
        dcc.Graph(id='graph3')
    ])
])


# define callbacks for updating graphs
@app.callback(
    Output('graph1', 'figure'),
    [Input('cohort-dropdown', 'value'),
     Input('feature-dropdown', 'value')])
def update_graph1(cohort, feature):
    df_cohort = df.loc[df.cohort == cohort]
    df_monthly = df_cohort.groupby(pd.Grouper(key='date', freq='M')).total_spend.sum().reset_index()
    fig = px.bar(
        df_monthly,
        x='date',
        y='total_spend',
        color='cohort',
        barmode='group',
        title='Total Spend per Month for each Cohort'
    )
    return fig


@app.callback(
    Output('graph2', 'figure'),
    [Input('cohort-dropdown', 'value')])
def update_graph2(cohort):
    df_cohort = df.loc[df.cohort == cohort]
    fig = px.line(
        df_cohort,
        x='date',
        y=['conversion_per_session', 'price_per_impression'],
        color='cohort',
        title='Conversion per session vs Price per impression for  each Cohort'
    )
    return fig


@app.callback(
    Output('graph3', 'figure'),
    [Input('cohort-dropdown', 'value')])
def update_graph3(cohort):
    df_cohort = df.loc[df.cohort == cohort]
    fig = px.bar(
        df_cohort,
        x='date',
        y='CPClick',
        color='cohort',
        title=' Cost per click  each Cohort'
    )
    return fig


# run app
if __name__ == '__main__':
    app.run_server(debug=True)



Dash is running on http://127.0.0.1:8051/

 * Serving Flask app '__main__'
 * Debug mode: off
